In [1]:
import os
import ee
import ipyleaflet
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import statsmodels.api as sm
import pandas as pd
from IPython.display import HTML, display
import random
import json
num_seed=30
random.seed(num_seed)

In [2]:
#Define functions for mapping

def GetTileLayerUrl(eeImage):
    map_id = ee.Image(eeImage).getMapId()
    tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
    return tile_url_template.format(**map_id)

def TileLayer(eeImage, **kwargs):
    return ipyleaflet.TileLayer(url=GetTileLayerUrl(eeImage), **kwargs)

def Map(*args, **kwargs):
    if not kwargs:
        kwargs = {'center':(48.2, 16.3), 'zoom':2, 'layout':{'height':'200px'}}
    Map = ipyleaflet.Map(*args, **kwargs)
    Map.add_control(ipyleaflet.LayersControl())
    return Map

def flatten(imageCollection):
    return ee.Image(
        imageCollection.iterate(
            lambda im, prev: ee.Image(prev).addBands(im), ee.Image()
        )).slice(1)

def rollback(image):
    return image.addBands(image.select(0)).slice(1)

def roll(image):
    return image.slice(-1).addBands(image.slice(0,-1))

def reindex(image, pfx='b'):
    names = ee.List.sequence(0, count=image.bandNames().length()).map(
        lambda i: ee.String(pfx).cat(ee.Number(i).format('%d'))
    )
    return image.rename(names)


In [3]:

ee.Initialize()


In [4]:
#Define functions for mapping MapBiomas and simplifying the legend
coverage_palette =  ['ffffff', '129912', '1f4423', '006400', '00ff00', '687537', '76a5af', '29eee4', 
                     '77a605', '935132', 'bbfcac', '45c2a5', 'b8af4f', 'f1c232', 'ffffb2', 'ffd966', 
                     'f6b26b', 'f99f40', 'e974ed', 'd5a6bd', 'c27ba0', 'fff3bf', 'ea9999', 'dd7e6b', 
                     'aa0000', 'ff99ff', '0000ff', 'd5d5e5', 'dd497f', 'b2ae7c', 'af2a2a', '8a2be2', 
                     '968c46', '0000ff', '4fd3ff']


simple_palette = ['129912','BBFCAC','FFFFB2','EA9999','0000FF','D5D5E5']
statesViz = {'min': 0, 'max': 34, 'palette': coverage_palette};
simpleStatesViz = {'min': 1, 'max': 6, 'palette': simple_palette};

change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 0, 'max': 1, 'palette': change_detection_palette};


mapbiomas_states=ee.Image('projects/mapbiomas-workspace/public/collection4_1/mapbiomas_collection41_integration_v1')
states_mask = mapbiomas_states.mask()

def simplify_legend(bandName):
    simplify = mapbiomas_states.expression(
        '(b0 >=1)  && (b0<10) ? 1 :'+
        '((b0>=10) && (b0<14)) || (b0==32) || (b0==29) ? 2 :'+
        '((b0>=18) && (b0<22)) || ((b0>=14)&&(b0<16)) ? 3 :'+
        '((b0>=22) && (b0<26)) || (b0==30) ? 4 :'+
        '(b0==26) || (b0==33) || (b0==31) ? 5 : 6', 
        {
          'b0': mapbiomas_states.select([bandName])
        })
    simplify = simplify.select(['constant'],[bandName])
    return simplify

bandList = ['classification_1985', 'classification_1986', 'classification_1987', 'classification_1988', 
             'classification_1989', 'classification_1990', 'classification_1991', 'classification_1992', 
             'classification_1993', 'classification_1994', 'classification_1995', 'classification_1996', 
             'classification_1997', 'classification_1998', 'classification_1999', 'classification_2000', 
             'classification_2001', 'classification_2002', 'classification_2003', 'classification_2004', 
             'classification_2005', 'classification_2006', 'classification_2007', 'classification_2008', 
             'classification_2009', 'classification_2010', 'classification_2011', 'classification_2012', 
             'classification_2013', 'classification_2014', 'classification_2015', 'classification_2016', 
             'classification_2017', 'classification_2018']
bandsEEList = ee.List(bandList) 
states_simple = ee.ImageCollection(bandsEEList.map(simplify_legend)).toBands()
states_simple = states_simple.select(states_simple.bandNames(),bandsEEList)
states_simple = states_simple.updateMask(states_mask)

m = Map(zoom=4, center=(-9, -51))

m.add_layer(TileLayer(mapbiomas_states.select('classification_2018').visualize(**statesViz), name='Original MapBiomas'))
m.add_layer(TileLayer(states_simple.select('classification_2018').visualize(**simpleStatesViz), name='Simplified MapBiomas'))

display(m)


Map(center=[-9, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [5]:
#Convert long band names to short band names
intBandNames = ['1985', '1986', '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995', '1996', 
             '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',  '2005', '2006', '2007', '2008', 
             '2009', '2010', '2011', '2012','2013', '2014', '2015', '2016', '2017', '2018']

int_states_simple = states_simple.select(bandList,intBandNames)

def convert_to_image_collection(bandName):
    return states_simple.select(bandName)


In [6]:
#Define functions for finding different types of change in the MapBiomas series
def lc_one_change(bandName):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    return int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))

def lc_no_change(bandName):
    '''
    Determines if there was one change occurance from year i to year i+1. Returns an image with values:
    1 if state(i) != state(i+1)
    0 if state(i) == state(i+1)
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    return int_states_simple.select([currentYearString]).eq(int_states_simple.select([nextYearString]))


def lc_reverse(bandName):
    '''
    Determines if change that occured from i to i+1 reversed back to state i in i+2
    1 if state(i) != state(i+1) and state(i) == state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    returnback = int_states_simple.select([currentYearString]).eq(int_states_simple.select([nextNextYearString]))
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    lc_reversed = returnback.bitwise_and(changed)
    return lc_reversed

def lc_change_to_another(bandName):
    '''
    Determines if change that occured from i to i+1 reversed back to state i in i+2
    1 if state(i) != state(i+1) and state(i) != state(i+2) and state(i+1) != state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    changed_again = int_states_simple.select([nextYearString]).neq(int_states_simple.select([nextNextYearString]))
    not_reversed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextNextYearString]))

    lc_changed_to_another = changed.bitwise_and(changed_again.bitwise_and(not_reversed))
    return lc_changed_to_another


def lc_consistent_change_one_year(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2
    1 if state(i) != state(i+1) and state(i+1) == state(i+2)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed)
    return lc_consistently_changed

def lc_consistent_change_two_years(bandName):
    '''
    Determines if change that occured from i to i+1 stayed in i+2 and i+3
    1 if state(i) != state(i+1) and state(i+1) == state(i+2) and state(i+1) == state(i+3)
    0 otherwise
    '''
    currentYear = ee.Number(bandName)
    nextYear = currentYear.add(ee.Number(1))
    nextNextYear = currentYear.add(ee.Number(2))
    nextNextNextYear = currentYear.add(ee.Number(3))
    
    currentYearString = ee.List(intBandNames).get(currentYear)
    nextYearString = ee.List(intBandNames).get(nextYear)
    nextNextYearString = ee.List(intBandNames).get(nextNextYear)
    nextNextNextYearString = ee.List(intBandNames).get(nextNextNextYear)
    
    changed = int_states_simple.select([currentYearString]).neq(int_states_simple.select([nextYearString]))
    stayed_one_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextYearString]))
    stayed_two_year = int_states_simple.select([nextYearString]).eq(int_states_simple.select([nextNextNextYearString]))
    lc_consistently_changed = changed.bitwise_and(stayed_one_year.bitwise_and(stayed_two_year))
    return lc_consistently_changed


In [7]:
#Apply functions to MapBiomas image series
years_for_one_change = np.arange(0,len(intBandNames)-1).tolist()#
years_for_two_change = np.arange(0,len(intBandNames)-2).tolist()#
years_for_three_change = np.arange(0,len(intBandNames)-3).tolist()#

states_lc_no_change = ee.ImageCollection(ee.List(years_for_one_change).map(lc_no_change)).toBands()
states_lc_no_change = states_lc_no_change.select(states_lc_no_change.bandNames(),intBandNames[0:len(intBandNames)-1])

states_lc_one_change = ee.ImageCollection(ee.List(years_for_one_change).map(lc_one_change)).toBands()
states_lc_one_change = states_lc_one_change.select(states_lc_one_change.bandNames(),intBandNames[0:len(intBandNames)-1])

states_lc_reverse = ee.ImageCollection(ee.List(years_for_two_change).map(lc_reverse)).toBands()
states_lc_reverse = states_lc_reverse.select(states_lc_reverse.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_change_to_another = ee.ImageCollection(ee.List(years_for_two_change).map(lc_change_to_another)).toBands()
states_lc_change_to_another = states_lc_change_to_another.select(states_lc_change_to_another.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_consistent_change_one_year = ee.ImageCollection(ee.List(years_for_two_change).map(lc_consistent_change_one_year)).toBands()
states_lc_consistent_change_one_year = states_lc_consistent_change_one_year.select(states_lc_consistent_change_one_year.bandNames(),intBandNames[0:len(intBandNames)-2])

states_lc_consistent_change_two_years = ee.ImageCollection(ee.List(years_for_three_change).map(lc_consistent_change_two_years)).toBands()
states_lc_consistent_change_two_years = states_lc_consistent_change_two_years.select(states_lc_consistent_change_two_years.bandNames(),intBandNames[0:len(intBandNames)-3])



In [8]:
#Get mask of when tiles had no change
change_occured = states_lc_one_change.reduce(ee.Reducer.max())


In [9]:
m_change_detection = Map(zoom=4, center=(-9, -51))

m_change_detection.add_layer(TileLayer(states_lc_consistent_change_two_years.select('2015').visualize(**changeDetectionViz), name='One Change'))
#m.add_layer(TileLayer(states_simple.select('classification_2018').visualize(**simpleStatesViz), name='Simplified MapBiomas'))

display(m_change_detection)
#Pink is no change
#Blue is change

Map(center=[-9, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [10]:
#Sample points in Brazil

Brazil_adm1 = ee.FeatureCollection('users/listerkristineanne/DynamicWorld/ChangeDetection/MapBiomas/Brazil_adm1')
#print(Brazil_adm1.aggregate_array('NAME_1').getInfo())


consistent_change_occurred = states_lc_consistent_change_two_years.reduce(ee.Reducer.max())


print(consistent_change_occurred.getInfo())
m_change_detection2 = Map(zoom=4, center=(-9, -51))

m_change_detection2.add_layer(TileLayer(consistent_change_occurred.visualize(**changeDetectionViz), name='One Change'))

display(m_change_detection2)
#Pink is no change
#Blue is change

def feature_calc(feature):
    feature = ee.Feature(feature)
    meanDictionary = consistent_change_occurred.reduceRegion(
      reducer= ee.Reducer.sum().unweighted(),
      geometry= feature.geometry(),
      scale= 30,
      maxPixels= 1e13)
    return ee.Feature(None,meanDictionary)

results = Brazil_adm1.map(feature_calc)

export_results_task2 = ee.batch.Export.table.toDrive(
    collection=results, 
    description = "Sum results", 
    fileNamePrefix = 'SumResults')

export_results_task2.start()



{'type': 'Image', 'bands': [{'id': 'max', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


Map(center=[-9, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [11]:
consistent_change_occurred_masked = consistent_change_occurred.add(1)
consistent_change_occurred_masked = consistent_change_occurred_masked.updateMask(change_occured)


m_change_detection3 = Map(zoom=4, center=(-9, -51))


change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 1, 'max': 2, 'palette': change_detection_palette};


m_change_detection3.add_layer(TileLayer(consistent_change_occurred_masked.visualize(**changeDetectionViz), name='One Change'))

display(m_change_detection3)


Map(center=[-9, -51], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_te…

In [13]:
# Get neighborhood values for stratified random sample

def getSampleBandPoints(image, region, **kwargs):
    dargs = {
        'numPoints':100,
        'region':region,
        'classValues':[0,1], 
        'classPoints':[0,100],
        'seed':794274,
        'geometries':True,
        'scale': image.projection().nominalScale()
    }
    dargs.update(kwargs)
    bandSamples = image.bandNames().map(lambda b:
        image.select(ee.String(b)).stratifiedSample(
            **dargs),
        )
    return bandSamples



def getSampleBandData(image, sampleBandPoints):
    idxIter = ee.List.sequence(0, count=sampleBandPoints.length())
    sampleBandData = idxIter.map(lambda i:
        image.select(ee.Number(i)).reproject(image.projection()).reduceRegions(
            collection=sampleBandPoints.get(ee.Number(i)),
            reducer=ee.Reducer.first(),
            scale = image.projection().nominalScale()
        ))
    return sampleBandDataToList(sampleBandData)

def get_point_coordinates(sampleBandPoints):
    return ee.FeatureCollection(sampleBandPoints).flatten().geometry().coordinates().getInfo()
    
    
def sampleBandDataToList(sampleData):
    return ee.FeatureCollection(sampleData).flatten().aggregate_array('first')


sample_points = getSampleBandPoints(consistent_change_occurred_masked, Brazil_adm1.geometry(),classValues=[1,2],classPoints=[200,200])


In [24]:
m_change_detection4 = Map(zoom=4, center=(-9, -51))


change_detection_palette = ['df07b5','0741df']
changeDetectionViz = {'min': 1, 'max': 2, 'palette': change_detection_palette};
print(sample_points.getInfo())
sample_points_image = ee.Image().paint(**{
    'featureCollection': sample_points,
    'color': 0,
    'width': 3
})
print(sample_points_image.getInfo())
# sample_points_palette = ['0cf910']
# sample_points_Viz = {'min': 1, 'max': 1, 'palette': sample_points_palette};


# m_change_detection4.add_layer(TileLayer(consistent_change_occurred_masked.visualize(**changeDetectionViz), name='One Change'))
# m_change_detection4.add_layer(TileLayer(sample_points_image.visualize(**sample_points_Viz), name='Sample Points'))

# display(m_change_detection4)


[{'type': 'FeatureCollection', 'columns': {'latitude_209564535': '<any>', 'longitude_209564535': '<any>', 'max': 'Byte<1, 2>'}, 'properties': {'band_order': ['max']}}]
{'type': 'Image', 'bands': [{'id': 'constant', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 255}, 'crs': 'EPSG:4326', 'crs_transform': [1.0, 0.0, 0.0, 0.0, 1.0, 0.0]}]}


In [ ]:
full_res = {}

coords = ee.Dictionary({'coordinates':get_point_coordinates(sample_points)}).getInfo()

full_res.update(coords)

for i in years_for_three_change:
    bandName = intBandNames[i]
    res = ee.Dictionary({
    'one_change_'+bandName: getSampleBandData(states_lc_one_change.select([bandName]), sample_points),
    #'reverse_change_'+bandName: getSampleBandData(states_lc_reverse.select([bandName]), sample_points),
    #'change_to_another_'+bandName: getSampleBandData(states_lc_change_to_another.select([bandName]), sample_points),
    #'consistent_change_one_year_'+bandName: getSampleBandData(states_lc_consistent_change_one_year.select([bandName]), sample_points),
    'consistent_change_two_years_'+bandName: getSampleBandData(states_lc_consistent_change_two_years.select([bandName]), sample_points),
    }).getInfo()
    full_res.update(res)

# res = ee.Dictionary({
#     'one_change': getSampleBandData(states_lc_one_change, sample_points),
#     'reverse_change': getSampleBandData(states_lc_reverse, sample_points),
#     'change_to_another': getSampleBandData(states_lc_change_to_another, sample_points),
#     'consistent_change_one_year': getSampleBandData(states_lc_consistent_change_one_year, sample_points),
#     'consistent_change_two_years': getSampleBandData(states_lc_consistent_change_two_years, sample_points),
# }).getInfo()
#ordered = ['one_change', 'reverse_change', 'change_to_another', 'consistent_change_one_year','consistent_change_two_years']
ordered = ['one_change','consistent_change_two_years']
results_df = pd.DataFrame(full_res)

results_df.to_csv('/Users/kristine/Downloads/NewSampledPoints.csv')


In [ ]:
results_df

In [ ]:
results_df.to_csv('/Users/kristine/Downloads/NewSampledPoints.csv')


In [ ]:
def convert_wideform_to_longform(in_df,years,id_column = 'one_change'):
    for i,year in enumerate(years):
        match_columns = [x for x in list(in_df) if str(year) in x]
        

In [ ]:
# sampled_points = consistent_change_occurred.stratifiedSample(
#                 numPoints=500,
#                 region=Brazil_adm1.geometry(),
#                 seed=num_seed,
#                 scale = consistent_change_occurred.projection().nominalScale(),
#                 geometries=True)

# print(sampled_points.size().getInfo())

# sampled_points = states_lc_one_change.reduceRegions(sampled_points, ee.Reducer.first());

# sample_points_task = ee.batch.Export.table.toDrive(
#     collection = sampled_points, 
#     description = "Sampled Points", 
#     fileNamePrefix = 'SamplePoints')

# sample_points_task.start()

In [ ]:
# #Read in sampled points from file downloaded from google drive
# df = pd.read_csv('/Users/kristine/Downloads/SamplePoints.csv')
# df = df[['max','.geo']]
# df.columns = ['max','coordinates']

# for i,row in df.iterrows():
#     df.at[i,'coordinates']  = json.loads(row['coordinates']).get('coordinates')
# print(df)

#convert to feature collection

In [ ]:
#print(sampled_points.geometry().coordinates().getInfo())